# 🦷 Huấn luyện mô hình Object Detection cho răng sâu trên Google Colab

**Lưu ý**: Bạn có thể download dataset Oral Diseases từ Kaggle trực tiếp trên Colab.

## 0. Download Dataset từ Kaggle (Tùy chọn)

In [ ]:
# Download dataset Oral Diseases từ Kaggle (KHÔNG CẦN GOOGLE DRIVE)
!pip install kagglehub

import kagglehub
import os
import shutil
from pathlib import Path

# Download latest version
print("Đang download dataset Oral Diseases từ Kaggle...")
path = kagglehub.dataset_download("salmansajid05/oral-diseases")
print("Path to dataset files:", path)

# Kiểm tra và copy dataset
if os.path.exists(path):
    dataset_name = "oral_diseases"
    if os.path.exists(dataset_name):
        shutil.rmtree(dataset_name)
    shutil.copytree(path, dataset_name)
    print(f"Đã copy dataset vào thư mục: {dataset_name}")

    print(f"\nCấu trúc dataset {dataset_name}:")
    for root, dirs, files in os.walk(dataset_name):
        level = root.replace(dataset_name, '').count(os.sep)
        indent = ' ' * 2 * level
        if level == 0:
            print(f"{indent}{os.path.basename(root)}/")
        else:
            print(f"{indent}{os.path.basename(root)}/")
        if level < 2:
            subindent = ' ' * 2 * (level + 1)
            for file in files[:2]:
                print(f"{subindent}{file}")
            if len(files) > 2:
                print(f"{subindent}... và {len(files)-2} file khác")

# 🦷 Huấn luyện mô hình Object Detection cho răng sâu trên Google Colab

Notebook này sẽ hướng dẫn bạn train mô hình YOLOv8 để phát hiện và khoanh vùng răng sâu trên ảnh.

## 1. Cài đặt thư viện

In [ ]:
# Kết nối Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Cài đặt YOLOv8
!pip install ultralytics
!pip install opencv-python matplotlib

## 2. Import và cấu hình

In [ ]:
from ultralytics import YOLO
import os
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

# Cấu hình đường dẫn
DATA_DIR = '/content/drive/MyDrive/dental_detection_data'
MODEL_SAVE_PATH = '/content/drive/MyDrive/dental_detection_model.pt'

print(f"Data directory: {DATA_DIR}")
print(f"Model will be saved to: {MODEL_SAVE_PATH}")

In [ ]:
# Cấu hình đường dẫn
# Dataset đã download từ Kaggle:
DATA_DIR = '/content/oral_diseases'  # Dataset đã download từ Kaggle

# Lưu model trực tiếp trên Colab:
MODEL_SAVE_PATH = '/content/dental_detection_model.pt'

print(f"Data directory: {DATA_DIR}")
print(f"Model will be saved to: {MODEL_SAVE_PATH}")

# Kiểm tra dataset
if os.path.exists(DATA_DIR):
    print(f"✅ Dataset tồn tại tại: {DATA_DIR}")
    # Kiểm tra cấu trúc cho YOLO
    images_train = os.path.join(DATA_DIR, 'images', 'train')
    images_val = os.path.join(DATA_DIR, 'images', 'val')
    labels_train = os.path.join(DATA_DIR, 'labels', 'train')
    labels_val = os.path.join(DATA_DIR, 'labels', 'val')

    if os.path.exists(images_train) and os.path.exists(labels_train):
        print("✅ Dataset đã có cấu trúc YOLO chuẩn")
    else:
        print("⚠️  Dataset chưa có cấu trúc YOLO chuẩn")
        print("Bạn cần tổ chức dataset theo format:")
        print("oral_diseases/")
        print("├── images/train/     # Ảnh training")
        print("├── images/val/       # Ảnh validation")
        print("├── labels/train/     # File .txt annotations")
        print("└── labels/val/       # File .txt annotations")
else:
    print(f"❌ Dataset không tồn tại tại: {DATA_DIR}")
    print("Hãy download dataset trước!")

## 3. Chuẩn bị dataset và config

In [ ]:
# Tạo file config YAML cho YOLO
yaml_content = f"""
path: {DATA_DIR}
train: images/train
val: images/val

names:
  0: cavity
"""

yaml_path = '/content/data.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("Đã tạo file config data.yaml")
print("Nội dung:")
print(yaml_content)

## 4. Huấn luyện mô hình YOLOv8

In [ ]:
# Load mô hình YOLOv8
model = YOLO('yolov8n.pt')  # Sử dụng model nano cho tốc độ nhanh

# Huấn luyện mô hình
results = model.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,
    name='dental_cavity_detection',
    save=True,
    save_period=10,
    cache=False,
    device=0  # Sử dụng GPU
)

print("Hoàn thành training!")

## 5. Test và đánh giá mô hình

In [ ]:
# Load best model
best_model_path = 'runs/detect/dental_cavity_detection/weights/best.pt'
best_model = YOLO(best_model_path)

# Test trên ảnh mẫu
test_images = list(Path(f"{DATA_DIR}/images/val").glob("*.jpg"))[:5]

for img_path in test_images:
    results = best_model(img_path)
    results[0].show()
    print(f"Đã xử lý: {img_path.name}")

print("Hoàn thành test!")

## 6. Lưu và export mô hình

In [ ]:
# Copy best model to current directory
source_path = 'runs/detect/dental_cavity_detection/weights/best.pt'
if os.path.exists(source_path):
    shutil.copy2(source_path, MODEL_SAVE_PATH)
    print(f"Đã lưu mô hình tại: {MODEL_SAVE_PATH}")

    # Kiểm tra kích thước file
    size_mb = os.path.getsize(MODEL_SAVE_PATH) / (1024 * 1024)
    print(".2f")
else:
    print("Không tìm thấy file best.pt")

print("Training hoàn tất!")
print("\n📥 DOWNLOAD MODEL:")
print("- Click vào file dental_detection_model.pt ở sidebar bên trái")
print("- Right-click > Download để tải về máy")
print("- Copy vào thư mục models/ của project local")